In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import pickle
from torch.nn.functional import pad, one_hot

from torch.utils.data import DataLoader
from matplotlib import pyplot as plt

import Transformer
from Transformer import TransformerClassifier, MyDataset, create_padding_mask

Combine smaller files

In [2]:
# ins = []
# for i in range (1,16):
#     name = "Train_Data_50_" +str(i)
#     with open(name,"rb") as file:
#         ins.append(pickle.load(file))

# out = pd.concat([x for x in ins])

# with open("Train_Data_m","wb") as file:
#         pickle.dump(out,file)

In [3]:
# name = "Train_Data_m"
# with open(name,"rb") as file:
#     raw_data = pickle.load(file)

# raw_data.shape

Create DataLoader

In [4]:
# Import Data
name = "Train_Data_m"
with open(name,"rb") as file:
    raw_data = pickle.load(file)


# Pad Data
data = raw_data["System_EWS"]
seq = [torch.from_numpy(run).float() for run in data]
#seq_trun = [torch.from_numpy(run[:,:6]).float() for run in data]
seq_padded = []
max_length = 650

for run in seq:
    pad_amount = max_length - run.shape[0]
    run_padded = pad(run, (0, 0, pad_amount, 0))
    seq_padded.append(run_padded)

input_data = torch.stack(seq_padded)

# Create one-hot labels
labels = [torch.tensor(label) for label in raw_data["null"]]
labels_t = torch.stack(labels)
labels_oh = one_hot(labels_t).float()


dataset = MyDataset(input_data, labels_oh)
batch_size = int(len(raw_data)/45)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [5]:
input_data.shape

torch.Size([900, 650, 12])

Import Test Data

Initialize Model and Select Parameters

In [6]:
dim_feedforward = 64
input_dim = 12  # Number of features
d_model = 64  # Transformer model dimension
num_heads = 4   # Number of attention heads
num_layers = 6  # Number of transformer layers
num_classes = 2  # Number of classes for classification
dropout = 0.15   # Dropout rate

model = TransformerClassifier(input_dim, d_model, num_heads, num_layers, num_classes, dim_feedforward, dropout)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [7]:
print(model)

TransformerClassifier(
  (batch_norm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (projection): Linear(in_features=12, out_features=64, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.15, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=64, bias=True)
        (dropout): Dropout(p=0.15, inplace=False)
        (linear2): Linear(in_features=64, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.15, inplace=False)
        (dropout2): Dropout(p=0.15, inplace=False)
      )
    )
  )
  (classificati

Train Model

In [8]:
epochs = 10 # number of epochs

losses = []

for epoch in range(1,epochs+1):
    for batch_data, batch_labels in dataloader:
        # Forward pass with masking
        output = model(batch_data)
        # print(f"output shape: {output.shape},output: {output}")  # Output will have shape: [batch_size, num_classes]
        # print(f"labels: {batch_labels}")
        # Example of using CrossEntropyLoss for training
        loss = criterion(output, batch_labels)
        print(f"Loss: {loss.item()}")
        losses.append(loss.item())
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()

Loss: 0.7098137140274048
Loss: 0.8039897084236145
Loss: 1.2751307487487793
Loss: 0.9281856417655945
Loss: 0.7069808840751648
Loss: 0.7660684585571289
Loss: 0.5746616721153259
Loss: 0.5057948231697083
Loss: 0.43982839584350586
Loss: 0.6874915957450867
Loss: 0.6175878643989563
Loss: 0.5559170842170715
Loss: 0.49956855177879333
Loss: 0.4539553225040436
Loss: 0.5956739187240601
Loss: 0.4947638511657715
Loss: 0.34435293078422546
Loss: 0.4547487795352936
Loss: 0.41061314940452576
Loss: 0.4669405519962311
Loss: 0.38921618461608887
Loss: 0.3318195939064026
Loss: 0.3657333254814148
Loss: 0.39640748500823975
Loss: 0.5566005706787109
Loss: 0.5884007215499878
Loss: 0.41651278734207153
Loss: 0.44786104559898376
Loss: 0.47292953729629517
Loss: 0.3076580762863159
Loss: 0.51820969581604
Loss: 0.5016682147979736
Loss: 0.45421963930130005
Loss: 0.3093903660774231
Loss: 0.41842183470726013
Loss: 0.42278680205345154
Loss: 0.4157359004020691
Loss: 0.21435561776161194
Loss: 0.4547498822212219
Loss: 0.380882